***ETF Evaluation***
********************************************************************************
**This script aims to evaluate, based on the hystorical date of an ETF, how much is convenient to reduce the occurence of the investment in the ETF**

***Libraries Definition***

In [2]:
##Generic library for Array and Data-time format
import datetime as dt
import time
import glob
import math
import os
import numpy as np
import pandas as pd

##Generic library to create plots
import plotly.graph_objects as go
import plotly.subplots as sp

##Generic library to retrieve ETF-Data
import yfinance as yf

Here we have the function that retrieve the ETF information based on the choosen ISIN

In [34]:
##Return the DATA INFORMATIONS
def month_year():
    now = dt.datetime.now()
    return now.day, now.month, now.year


##Return the yfinance.Ticker object that stores all the relevant ETF informations
def ETF_DATA_from_ISIN(isin_string):
    etf_ticker = isin_string
    etf_data = yf.Ticker(etf_ticker)
    etf_data.info
    return etf_data


##Dollar Cost Average Function
def calculate_average_cost(
    etf_data, initial_capital, start_date, end_date, purchase_frequency
):
    # Create a date range from start_date to end_date
    dates = pd.date_range(start_date, end_date, freq=purchase_frequency)
    # print(dates)
    # Initialize a list to store the purchase prices
    prices_list = []
    # Initialize a list to store the average costs
    average_cost_list = []
    # Initialize a list to store the amount of bought shares
    number_share_list = []
    # Initialize a list to store the amount of money spent
    total_purchase_amount_list = []
    # Initialize a list to store the date where I have bought
    dates_purchase_list = []
    # Initialization of some internal values used for computation
    total_purchase_amount = 0
    number_share = 0

    # Loop through each date in the date range
    for i, date in enumerate(dates):
        # Assume we're buying a fixed amount of the investment each time
        purchase_amount = initial_capital / len(dates)
        total_purchase_amount += purchase_amount
        # Collect the cost of the total investiment
        total_purchase_amount_list.append(total_purchase_amount)
        # print(f"total_purchase_amount {total_purchase_amount_list[-1]}", f"Date {date}")
        # Get the price of the investment on this date (e.g. from a database or API)
        # date must be a string
        date = date.strftime("%Y-%m-%d")
        price = get_price_etf(etf_data, date)
        # Collect date of each purchase
        dates_purchase_list.append(date)
        # Collect the cost per share for each purchase
        prices_list.append(price)
        # print(f"Price {prices_list[-1]}", f"Date {date}")
        # Calculate the number of shares
        number_share += purchase_amount / price
        number_share_list.append(number_share)
        # print(f"number_share {number_share_list[-1]}", f"Date {date}")
        # Calculate the average cost up to this point
        average_cost = total_purchase_amount / number_share
        # Add the date and average cost to the list of average costs
        average_cost_list.append(average_cost)
        # Calculate the final average cost
        final_average_cost = average_cost_list[-1]

    return (
        average_cost_list,
        final_average_cost,
        number_share_list,
        total_purchase_amount_list,
        dates_purchase_list,
    )


def get_price_etf(etf_data, date, timeout=90):
    start_time = time.time()
    while time.time() - start_time < timeout:
        if f"{date}" in etf_data.index:
            price_date = etf_data.loc[f"{date}"]
            return price_date
        else:
            # If no price data is found, increment the date by one day
            date = pd.to_datetime(date) + pd.Timedelta(days=1)
            date = date.strftime("%Y-%m-%d")
            time.sleep(1)  # wait for 1 second before trying again
    # If timeout is reached, raise an error
    raise ValueError(
        f"Failed to find price data for {etf_data.isin} on {date} after {timeout} seconds"
    )


##To be used in order to plot the ETF behavior along two dates that you choose
def plot_etf_data(etf_data, isin_string, start_date, end_date):
    try:
        hist_data = etf_data.history(start=start_date, end=end_date)
        fig = go.Figure(data=[go.Scatter(x=hist_data.index, y=hist_data["Close"])])
        fig.update_layout(
            title=f"ETF <b>{isin_string}</b> Price History",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
        )
        fig.show()
        hist_data = hist_data["Close"]
        hist_data.index = hist_data.index.strftime("%Y-%m-%d")
        return hist_data
    except Exception as e:
        print(f"Error: {e}")
        return None

GENERIC PLOT FUNCTION
===========================

In [29]:
def create_plot(x, y, name_trace, name_graph):
    fig = go.Figure()
    for x_list, y_list, name in zip(x, y, name_trace):
        fig.add_trace(
            go.Scatter(
                x=x_list,
                y=y_list,
                mode="lines+markers",
                name=name,
            )
        )
    fig.update_layout(
        title=name_graph,
        xaxis_title="",
        yaxis_title="",
        legend_title="Legenda",
        hovermode="x",
    )
    fig.show()
    return fig

***MAIN CODE***
===========================

Hystorical Behavior of the Selected ETF
===========================

In [5]:
### GENERIC INFORMATION ABOUT ETF and its HYSTORICAL BEHAVIOR
isin_string = "IE00B4L5Y983"
# Time Informations
day, month, year = month_year()
start_date = "2020-12-29"  ## %Y-%m-%d
end_date = f"{year}-{month}-{day}"
etf_data_object = ETF_DATA_from_ISIN(isin_string)
etf_data_daily_values = plot_etf_data(
    etf_data_object, isin_string, start_date, end_date
)

Evaluation with DCA with Custom Purchase Frequency
===========================

In [35]:
initial_capital = 10000
start_date_dca = "2020-12-29"
end_date_dca = "2024-05-01"
purchase_frequencies = ["3M", "6M", "12M", "24M"]  # range of purchase frequencies
results = {}

for freq in purchase_frequencies:
    (
        average_cost_list,
        final_average_cost,
        number_share_list,
        total_purchase_amount_list,
        dates_purchase_list,
    ) = calculate_average_cost(
        etf_data_daily_values,
        initial_capital,
        start_date_dca,
        end_date_dca,
        freq,
    )
    results[freq] = {
        "average_cost_list": average_cost_list,
        "final_average_cost": final_average_cost,
        "number_share_list": number_share_list,
        "total_purchase_amount_list": total_purchase_amount_list,
        "dates_purchase_list": dates_purchase_list,
    }
    ##Calculation about the gain

C:\Users\Davidde\AppData\Local\Temp\ipykernel_8136\1017211275.py:20: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



Plotting Section for DCA vs Sum Lump 
====================================

In [38]:
y_list_fig = [etf_data_daily_values] + [
    result["average_cost_list"] for result in results.values()
]
x_list_fig = [etf_data_daily_values.index] + [
    result["dates_purchase_list"] for result in results.values()
]
name_trace_list_fig = ["Indice Tracciato"] + [
    f"Costo di Carico Medio {freq}" for freq in purchase_frequencies
]
fig_dca = create_plot(
    x=x_list_fig,
    y=y_list_fig,
    name_graph="Grafico Acquisto con DCA",
    name_trace=name_trace_list_fig,
)
##Qui nuovi plot per il guadagno